In [1]:
!pip install selenium

!pip install beautifulsoup4

In [1]:
#Задача:
#1)Использовать прокси, для обхода блокировки
#2)Залогиниться для получения информации профиля
#3)Получить в профиле нужную страницу для ее парсинга
#4)Использовать скрипт для скроллинга, учитывая пагинацию работает подгружаемым JS
#5)Получить данные: Имя, компания, должность, мэил
#5)Для получения данных  необходимо: Пройти каждому из пользователей в профиль, открыть эмулятором браузера подгружаемое JS окно с контактными данными
#6)Произвести итерации с этими же действиями, по каждому из пользователей 
import requests, time, random
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

In [2]:
with open('linkedin.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['url','Имя','Компания', 'Должность', 'Email', 'Адрес'])

In [12]:
#Устанавливаем прокси для обхода блокировки, под проксей открываем страницу "входа", для ввода логина и праоля
PROXY = '51.38.71.101:8080'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

chrome = webdriver.Chrome(chrome_options=chrome_options, executable_path='C:/Users/Денис/Desktop/linkedin/drive/chromedriver.exe')
chrome.get("https://www.linkedin.com/uas/login")

c:\users\денис\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [4]:
#Открываем файл "config.txt", читаем, по ключам, то, что записали в него
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [5]:
#Находим элемент по id и методом send_keys, вводим логин- записав в аргумент ранее ссылающую по ключу переменную
elementID = chrome.find_element_by_id('username')
elementID.send_keys(username)

In [6]:
#Находим элемент по id и методом send_keys, вводим пароль- записав в аргумент ранее ссылающую по ключу переменную
elementID = chrome.find_element_by_id('password')
elementID.send_keys(password)

In [7]:
#Нажимем кнопку входа
elementID.submit()

In [8]:
#Обозначаем стартовую страницу, с необходимой информацией для парсинга
visiting_profiled_id = '/mynetwork/invite-connect/connections/'
full_link = 'https://www.linkedin.com' + visiting_profiled_id
chrome.get(full_link)

In [9]:
#Получаем html страницы connections, разбираем его библиотекой BeautifulSoup
src = chrome.page_source
soup = BeautifulSoup(src, 'lxml')

In [10]:
#Получаем все теги li содержащие имя и урлы>итерируем каждый li получая из каждого из них лиинк и урл. Урлы записываем в список urls, конкатенируя href
urls = []
name_li = soup.find_all('li', {'class':'list-style-none'})
for a in name_li:    
    links = a.find('div', {'class':'mn-connection-card ember-view'}).find('div', {'class': 'mn-connection-card__details'}).find('a').get('href')
    urls.append('https://www.linkedin.com' + links)

In [11]:
#Записываем то, что спарсили в csv
def write_csv(data):
    with open('linkedin.csv', 'a', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([data['full_link_doc'],
                         data['name_of_connect_person'],
                         data['company_name'],
                         data['position_in_company_of_connect_person'],
                         data['email'],
                         data['location'],
                         data['mobile_phone'],
                         data['connected']])
        
#Зацикливаем полученый ранее список для получения информации со страницы у каждого из пользователей
for url in urls:
    visiting_profile = url
    chrome.get(visiting_profile)
    
    #Открываем контактную информацию, скролим, получаем хтмл
    chrome.find_element_by_link_text('Contact info').click()
    
    SCROLL_PAUSE_TIME = 5

    # Get scroll height
    last_height = chrome.execute_script("return document.body.scrollHeight")

    for i in range(3):
        # Scroll down to bottom
        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = chrome.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    src = chrome.page_source
    soup = BeautifulSoup(src, 'lxml')
    try:
        email = soup.find('section', {'class':'pv-contact-info__contact-type ci-email'}).find('div', {'class': 'pv-contact-info__ci-container t-14'}).find('a', {'class':'pv-contact-info__contact-link t-14 t-black t-normal'}).text.strip()
    except:
        email = 'empty'
    try:    
        full_link_doc = 'https://www.' + soup.find('a', {'class':'pv-contact-info__contact-link t-14 t-black t-normal'}).text.strip()
    except:
        full_link_doc = 'empty'
    try:
        mobile_phone = soup.find('section', {'class':'pv-contact-info__contact-type ci-phone'}).find('ul', {'class': 'list-style-none'}).find('span', {'class':'t-14 t-black t-normal'}).text.strip()
    except:
        mobile_phone = 'empty'
    try:
        connected = soup.find('section', {'class':'pv-contact-info__contact-type ci-connected'}).find('div', {'class': 'pv-contact-info__ci-container t-14'}).find('span', {'class':'pv-contact-info__contact-item t-14 t-black t-normal'}).text.strip()
    except:
        connected = 'empty'
        
    print(email, full_link_doc, mobile_phone, connected)

    chrome.get(visiting_profile)    
    
    # Тоже самое проделываем и с основной страницей полльзователя, собираем нужную информацию
    SCROLL_PAUSE_TIME = 5

    # Get scroll height
    last_height = chrome.execute_script("return document.body.scrollHeight")

    for i in range(3):
        # Scroll down to bottom
        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = chrome.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
        src = chrome.page_source
        soup = BeautifulSoup(src, 'lxml')
        try:
            name_of_connect_person = soup.find('ul', {'class': 'pv-top-card--list inline-flex align-items-center'}).find('li', {'class':'inline t-24 t-black t-normal break-words'}).text.strip()
        except: 
            name_of_connect_person = 'empty'
        try:    
            position_in_company_of_connect_person = soup.find('h2', {'class':'mt1 t-18 t-black t-normal'}).text.strip()
        except:
            position_in_company_of_connect_person = 'empty'
        try:
            location = soup.find('li', {'class': 't-16 t-black t-normal inline-block'}).text.strip()
        except:
            location = 'empty'
        try:
            company_name = soup.find('ul', {'class': 'pv-top-card--experience-list'}).find('span').text.strip()
        except:
            company_name = 'empty'
            
        data = {'full_link_doc': full_link_doc,
               'name_of_connect_person': name_of_connect_person,
               'company_name': company_name,
               'position_in_company_of_connect_person': position_in_company_of_connect_person,
               'email':email,
               'location': location,
               'mobile_phone': mobile_phone,
               'connected': connected}
        write_csv(data)
        
        
        

md.mehedihasan9589@gmail.com https://www.linkedin.com/in/md-mehedi-hasan-747b72120 +8801715599589 February 26, 2020
rohit.pr3612@gmail.com https://www.linkedin.com/in/rohit-sharma-199028116 8073983557 February 26, 2020
feyisola@encyl.com https://www.linkedin.com/in/feyisola-olusesi-24031919b empty February 26, 2020
ozgurtaskaya@gmail.com https://www.linkedin.com/in/ozgurtaskaya empty January 30, 2020
katariya.sanjay@gmail.com https://www.linkedin.com/in/katariyasanjay +919810741164 January 30, 2020
mihir.goradia1@gmail.com https://www.linkedin.com/in/mihir-goradia-1986331a empty January 30, 2020
hotmaleuser@outlook.com https://www.linkedin.com/in/estebandobronsky empty January 30, 2020
mac.dinesh1434@gmail.com https://www.linkedin.com/in/dinesh-kumar-b8190b48 empty January 30, 2020
giladfecher1@gmail.com https://www.linkedin.com/in/gilad-fecher-932b7132 empty January 30, 2020
suhas.burte143@gmail.com https://www.linkedin.com/in/suhas-burte-45791262 9405411143 January 30, 2020
fabio.dal

d@followdigits.com https://www.linkedin.com/in/dominik-jenner-98869815 empty December 24, 2019
priyaanka.ray@gmail.com https://www.linkedin.com/in/priyankarayk empty December 23, 2019
olgad_pegas@mail.ru https://www.linkedin.com/in/olga-dracheva-6013b285 +79811961008 December 17, 2019
chanhoe@puretechglobal.com https://www.linkedin.com/in/chan-hoe-leung-a3a5a3197 empty December 17, 2019
tim@phybbit.com https://www.linkedin.com/in/timintokyo empty December 17, 2019
puja.noida1994@gmail.com https://www.linkedin.com/in/pooja-verma-b0359816a www.apptmedia.com September 24, 2019
georgianamaria.florea@gmail.com https://www.linkedin.com/in/georgiana-florea-3503b7125 empty September 24, 2019


JavascriptException: Message: javascript error: Cannot read property 'scrollHeight' of null
  (Session info: chrome=80.0.3987.122)
